# Chapter 5: First-Class Functions

Functions in Python are first-class objects, meaning they can be

* created at runtime,

* assigned to a variable or element in a data structure,

* passed as an argument to a function,

* returned as the result of a function.

Having first-class functions enables programming in a functional style.

## Treating a Function Like an Object

The cells below show that `factorial` is an instance of the `function` class, e.g. we can read its `__doc__` attribute.

In [1]:
def factorial(n):
    """Returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [2]:
factorial.__doc__

'Returns n!'

In [7]:
print(type(factorial))

<class 'function'>


In [8]:
fact = factorial
fact(5)

120

## Higher-Order Functions

A function that takes a function as an argument or returns a function as a result is called a *higher-order function*, e.g. `sorted` which allows us to sort values using a one-argument function.

Some of the most well-known built-in higher-order functions are `map`, `reduce`, `filter` and `apply`. The latter is deprecated, the three preceding functions are in most cases obsolete due to the introduction of listcomps and genexps.

In [11]:
# These two are equivalent
maplist = list(map(factorial, range(5)))
listcomp = [factorial(i) for i in range(5)]

print(maplist)
print(listcomp)

[1, 1, 2, 6, 24]
[1, 1, 2, 6, 24]


`map` and `filter`return generators, so their direct substitute is a generator expression.

In [12]:
g1 = map(factorial, range(5))
g2 = (factorial(i) for i in range(5))

print(next(g1))
print(next(g2))

1
1


The `reduce` function was demoted from built-in to the `functools` module, and is largely replaced by the use of conditionals in listcomps. It's most common use, summation, has largely been replaced by the `sum` built-in.

In [15]:
from functools import reduce
from operator import add

reduce(add, range(5)) == sum(range(5))

True

Other reducing built-ins are `any(iterable)` and `all(iterable)`.

`any` returns `True` if any element of `iterable` is truthy.

`all` returns `True` if every element of `iterable` is truthy.

## Anonymous Functions

Anonymous functions, or `lambda` functions, are handy when we need to create small one-off functions.

The body of a `lambda` cannot make assignments or use any other Python statements, such as `while`, `try`, etc.

The best recommended of an anonymous function is in the context of an argument list.

In [17]:
# Returns list sorted by reversed spelling
fruits = ["apple", "banana", "cherry", "raspberry", "strawberry", "fig"]
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### Lundh's Lambda Refactoring Recipe

If an anonymous function is hard to understand:

1. Write a comment explaining what `lambda` does.

2. Think of a name that captures the essence of the `lambda`.

3. Convert `lambda` to a `def` statement using that name.

4. Remove the comment.

## The Seven Flavours of Callable Objects

The call operator `()` may be applied to other objects beyond user-defined functions.

To determine whether an object is callable, use the `callable()` built-in function.

The Python Data Model documentation lists seven callable types:

1. *User-defined functions*: Created with `def`/`lambda` expressions.

2. *Built-in functions*: A function implemented in C (for CPython), like `len`.

3. *Built-in methods*: Methods implemented in C, like `dict.get`.

4. *Methods*: Functions defined in the body of a class.

5. *Classes*: When invoked, a class runs its `__new__` method to create an instance, then `__init__` to initialise it before returning to caller. Because there is no `new` operator in Python, calling a class is like calling a function.

6. *Class instances*: If a class defines a `__call__` method, then its instances may be invoked as functions.

7. *Generator functions*: Functions or methods that use the `yield` keyword. When called, generator functions return a generator object.

## User-Defined Callable Types

Implementing a `__call__` instance method will make any (arbitrary) Python object behave as a function.

Implementing a `__call__` is an easy way to create function-like objects that have some internal stat that must be kept across invocations.

Consider the example below, where `BingoCage` builds an instance from any iterable. Calling the instance pops an item.

In [44]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("pick from empty BingoCage")
            
    def __call__(self):
        return self.pick()
    
items = (1, 2, 3)
bingo = BingoCage(items)

# Check that callable
print(callable(bingo))

# Use class method ...
bingo.pick()

True


1

In [45]:
# ... or treat as function
bingo()

3

## Function Introspection

We can check which attributes are specific to functions that are not found in generic Python user-defined objects.

See table 5-1 on **p. 154** for details.

In [48]:
class C: pass
def f(): pass
sorted(set(dir(f)) - set(dir(C)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## From Positional to Keyword-Only Arguments

`*` an`**` explode iterables and mappings into separate arguments when we call a function, providing a extremely flexible parameter handling mechanism.

Keyword-only arguments will never capture positional arguments.

To specify a keyword-only argument, name them after the argument prefixed with `*`.

If you don't want to support variable positional arguments, but still want keyword-only arguments, put a `*` by itself in the signature.

In [50]:
def f(a, *, b):
    return a, b

f(1, 2)

TypeError: f() takes 1 positional argument but 2 were given

In [51]:
f(1, b=2)

(1, 2)

## Retrieving Information About Parameters

This section is about function introspection.

Within a function object, the `__defaults__` attribute holds a tuple with the default values of postitional and keyword arguments.

The defaults for keyword-only arguments appear in `__kwdefaults__`.

The names of the arguments are found within thew `__code__` atrribute, which references a `code` object with many attributes of its own.

### `inspect`

The `inspect` module provides a framework for function introspection.

`inspect.signature` returns an `inspect.Signature` object, which has a `parameters` attribute that grants access to reading an ordered mapping of `inspect.Parameter` objects. 

Each `Parameter` instance has attributes such as `name`, `default` and `kind`. The special value `inspect._empty` denotes parameters with no default (because `None` is a popular default value).

The `kind` attribute holds one of five possible values from the `_ParameterKind` class:

* `POSITIONAL_OR_KEYWORD`: Parameter that may be passed as a positional or keyword argument (most Python function parameters)

* `VAR_POSTIONAL`: A `tuple` of positional parameters.

* `VAR_KEYWORD`: A `dict` of parameters.

* `KEYWORD_ONLY`: A keyword-only parameter.

* `POSITIONAL_ONLY`: A positional-only parameter (at time of writing unsupported by Python but exemplified by existing functions written in C).

Besides the above, `inspect.Parameter` objects have an `annotation` attribute containing function signature metadata (e.g. used in `mypy`).

An `inspect.Signature` object has a `bind` method that takes any number of arguments and binds them to the parameters in the signature.

In [129]:
import inspect

def f(a, b=2): pass
sig = inspect.signature(f)
for s in sig.parameters.values():
    fmt = f"{str(s.kind)}: {s.name} = {s.default}"
    print(fmt)
print()

def g(a, *, b=2): pass
sig = inspect.signature(g)
for s in sig.parameters.values():
    fmt = f"{str(s.kind)}: {s.name} = {s.default}"
    print(fmt)
print()

    
def h(*, a): pass
sig = inspect.signature(h)
for s in sig.parameters.values():
    fmt = f"{str(s.kind)}: {s.name} = {s.default}"
    print(fmt)
print()
    
    
def j(*args, **kwargs): pass
sig = inspect.signature(j)
for s in sig.parameters.values():
    fmt = f"{str(s.kind)}: {s.name} = {s.default}"
    print(fmt)
print()

POSITIONAL_OR_KEYWORD: a = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD: b = 2

POSITIONAL_OR_KEYWORD: a = <class 'inspect._empty'>
KEYWORD_ONLY: b = 2

KEYWORD_ONLY: a = <class 'inspect._empty'>

VAR_POSITIONAL: args = <class 'inspect._empty'>
VAR_KEYWORD: kwargs = <class 'inspect._empty'>



In [137]:
def f(a, b=2): pass
sig = inspect.signature(f)
my_args = {"a": 2, "b": 4}
bound_args = sig.bind(**my_args)

for name, value in bound_args.arguments.items():
    print(name, "=", value)
    
# This will fail, because a is a required argument
del(my_args["a"])
bound_args = sig.bind(**my_args)

a = 2
b = 4


TypeError: missing a required argument: 'a'

## Function Annotations

Python 3 provides syntax to attach metadata to the parameters of a function declaration and return its value.

Each argument in a function declaration may have an annotation expression preceded by `:`. 

If there is a default value, the annotation goes between the name and the `=` sign.

To annotate a return value, add `->` between the `)` and the `:` at the tail of the function declaration.

The expressions may be of any type. No processing is done with the annotations. They are merely stored in the `__annotations__` attribute of the function (a `dict`).

In [140]:
def f(a: int, b: "int > 0"=2): pass

In [141]:
f.__annotations__

{'a': int, 'b': 'int > 0'}

In [144]:
import inspect

sig = inspect.signature(f)
for param in sig.parameters.values():
    print(repr(param.annotation))

<class 'int'>
'int > 0'


## Packages for Functional Programming

### `operator` 

The `operator` module provides function equivalents for dozens of arithmetic operators (convenient in a functional programming approach), saving us the trouble of writing trivial `lambda` expressions.

In [146]:
from functools import reduce

# Without function from operator
def fact(n):
    return reduce(lambda a, b: a*b, range(1, n + 1))

# With
from operator import mul
def fact_op(n):
    return reduce(mul, range(1, n + 1))

assert fact(5) == fact_op(5)

`itemgetter` and `attrgetter` build custom functions to to pick items from sequences or read attributes from objects.

`itemgetter` supports mappings and any class that implements `__getitem__`.

`itemgetter` is typically used to sort tuples by value of a field.

In [158]:
from operator import itemgetter
stuff = [
    ("A", "Z", 1),
    ("C", "Y", 6),
    ("B", "M", 7),
]

# Sort the list by field 1
for s in sorted(stuff, key=itemgetter(1)):
    print(s)

('B', 'M', 7)
('C', 'Y', 6)
('A', 'Z', 1)


In [156]:
# Sort the list by field 2
for s in sorted(stuff, key=itemgetter(2)):
    print(s)

('A', 'Z', 1)
('C', 'Y', 6)
('B', 'M', 7)


If multiple values are passed, the `itemgetter` will build and return tuples with the extracted values.

In [157]:
# Pick field 1 and 0, build and return new tuple
get_stuff = itemgetter(1, 0)
for thing in stuff:
    print(get_stuff(thing))

('Z', 'A')
('Y', 'C')
('M', 'B')


`attrgetter` creates functions to extract object attributes by name. If passed several attribute names, `attrgetter` will return a tuple of values.

If any argument name to `attrgetter` contains a `.` (dot), it will navigate through nested objects to retrieve the attribute.

In [183]:
from collections import namedtuple

metro_data = [("Tokyo", "JP", 36.933, (35.689722, 139.691667))]

LatLong = namedtuple("LatLong", "lat long")
Metropolis = namedtuple("Metropolis", "name cc pop coord")

metro_areas = [
    Metropolis(name, cc, pop, LatLong(lat, long)) 
    for name, cc, pop, (lat, long)
    in metro_data
]

In [186]:
from operator import attrgetter

name_lat = attrgetter("name", "coord.lat")

for area in metro_areas:
    print(name_lat(area))

('Tokyo', 35.689722)


`methodcaller` returns a function that calls a method by name on the object given as an argument. See **p. 166** for details.

## Freezing Arguments with `functools.partial`

`functools.partial` is a higher-order functions that allows partial application of a function.

Given a function, a partial application produces a new callable with some of the arguments of the original function fixed.

`functools.partialmethod` does the same, but is designed to work with methods.